In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.colors
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse2_new import fitEllipseContour
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!
import copy
import os
import scipy.stats
import pickle

from sklearn.linear_model import LinearRegression

#os.environ['DISPLAY']=':0.0'


In [2]:
## The main code begins here....
home_dir = '/home11/grad/2020/ef935217/Research/Ellipses/'  # <----Change this 
lev_list = [10,50,30]
temp_lev = [850]
#contour_list = [31500,20000,23000]
contour_list = [30000,20000,23000]

write_out = True #write the ellipse data to a text file, True or False 

In [3]:
##Open temp related pickles
infile = open("save_time.p",'rb')
year_ref = pickle.load(infile)
infile.close()

infile = open("save.p",'rb')
temp_ref = pickle.load(infile)
infile.close()

infile = open("lats.p",'rb')
lats = pickle.load(infile)
infile.close()

infile = open("lons.p",'rb')
lons = pickle.load(infile)
infile.close()

In [4]:
#Open data related pickles
infile = open("ephi_ratio10_1.p", 'rb')
ephi_ratio10 = pickle.load(infile)
infile.close()
infile = open("ephi_wind10_1.p", 'rb') 
ephi_wind10 = pickle.load(infile)
infile.close()
infile = open("ephi_cenlat10_1.p", 'rb')
ephi_cenlat10 = pickle.load(infile)
infile.close()
infile = open("ephi_size10_1.p", 'rb')
ephi_size10 = pickle.load(infile)
infile.close()
infile = open("ephi10_1.p", 'rb')
ephi10 = pickle.load(infile)
infile.close()


infile = open("ephi_ratio50_1.p", 'rb')
ephi_ratio50 = pickle.load(infile)
infile.close()
infile = open("ephi_wind50_1.p", 'rb') 
ephi_wind50 = pickle.load(infile)
infile.close()
infile = open("ephi_cenlat50_1.p", 'rb')
ephi_cenlat50 = pickle.load(infile)
infile.close()
infile = open("ephi_size50_1.p", 'rb')
ephi_size50 = pickle.load(infile)
infile.close()
infile = open("ephi50_1.p", 'rb')
ephi50 = pickle.load(infile)
infile.close()

In [5]:
#Averaging the appropriate elliptcial diagnostic
print(ephi10)
##10hPa
rat10 = np.nanmean(ephi_ratio10,axis=0)
wind10 = np.nanmean(ephi_wind10,axis=0)
cenl10 = np.nanmean(ephi_cenlat10,axis=0)
size10 = np.nanmean(ephi_size10,axis=0)
ep10 = np.nanmean(ephi10,axis=0)

##50hPa
rat50 = np.nanmean(ephi_ratio50,axis=0)
wind50 = np.nanmean(ephi_wind50,axis=0)
cenl50 = np.nanmean(ephi_cenlat50,axis=0)
size50 = np.nanmean(ephi_size50,axis=0)
ep50 = np.nanmean(ephi50,axis=0)

[[-35.01988139 -32.84623796 -29.57293755 ...  60.83180356  73.90323254
   41.79033504]
 [134.16348643 133.16544195 132.0364987  ...  63.11250075  72.02993405
   73.38380912]
 [ 92.85626251  93.91349119  94.665588   ...  66.28721792  71.12370793
   77.56052867]
 ...
 [-38.16347741 -38.00207472 -35.86590914 ... -43.31192652 -35.80228708
  -29.60951521]
 [104.25013655 104.10923817 103.17021375 ...          nan          nan
           nan]
 [106.38101474 107.33124577 108.68142734 ...  75.00747974  73.67545698
   71.92101465]]


In [6]:
## averaging temp values in grid
tempval = np.empty((20,488,64,134))
tempval[:] = np.nan

yearnot = 0

for i in np.arange(1999,2019): 
    #print(i)
    #print(yearnot)
    year_save = temp_ref[np.where(year_ref==i)]
    #print(year_save.shape)
    if i == 1999 or i == 2003 or i == 2007 or i == 2011 or i ==2015: #account for leap years in entering data
        tempval[yearnot,:,:,:] = year_save
        yearnot = yearnot+1
    else:
        tempval[yearnot,0:360,:,:] = year_save[0:360,:,:]
        tempval[yearnot,364:,:,:] = year_save[360:,:,:]
        yearnot = yearnot+1

print(tempval.shape)

tempvals = tempval[:,:,4:45,52:]
print(tempvals.shape)

temps = np.nanmean(tempvals,axis=0) #POR average temperatures 

print(temps.shape)

(20, 488, 64, 134)
(20, 488, 41, 82)
(488, 41, 82)


In [7]:
##SSW Averaging Timelines

##10hPa
ssw_wind10 = np.empty((12, 81))
ssw_wind10[:] = np.nan

ssw_rat10 = np.empty((12, 81))
ssw_rat10[:] = np.nan

ssw_size10 = np.empty((12, 81))
ssw_size10[:] = np.nan

ssw_cenl10 = np.empty((12, 81))
ssw_cenl10[:] = np.nan

ssw_ep10= np.empty((12, 81))
ssw_ep10[:] = np.nan

ssw_t= np.empty((12,81,41,82))
ssw_t[:] = np.nan


##50hPa
ssw_wind50 = np.empty((12, 81))
ssw_wind50[:] = np.nan

ssw_rat50 = np.empty((12, 81))
ssw_rat50[:] = np.nan

ssw_size50 = np.empty((12, 81))
ssw_size50[:] = np.nan

ssw_cenl50 = np.empty((12, 81))
ssw_cenl50[:] = np.nan

ssw_ep50= np.empty((12, 81))
ssw_ep50[:] = np.nan


year2 = [1999,2001,2002,2003,2005,2006,2007,2008,2009,2012,2017,2018]
sswcl2 = [0,2,3,4,6,7,8,9,10,13,18,19]
dates2 = [(404,485),(76,157),(156,237),(108,189),(168,249),(304,385),(296,377),(180,261),(244,325),(112,193),(256,337),(92,173)]

for i in range(len(year2)):
    d1 = int(dates2[i][0])
    d2 = int(dates2[i][1])
    
    ##diagnostics
    
    ##10hPa
    w = ephi_wind10[sswcl2[i],d1:d2] #wind
    ssw_wind10[i] = w[:]
    
    rat = ephi_ratio10[sswcl2[i],d1:d2]
    ssw_rat10[i] = rat[:]
    
    cl = ephi_cenlat10[sswcl2[i],d1:d2]
    ssw_cenl10[i] = cl[:]
    
    sz = ephi_size10[sswcl2[i],d1:d2]
    ssw_size10[i] = sz[:]
    
    ep = ephi10[sswcl2[i],d1:d2]
    ssw_ep10[i] = ep[:]
    
    tm = tempvals[sswcl2[i],d1:d2,:,:]
    ssw_t[i] = tm[:]
    
    ##50hPa
    w = ephi_wind50[sswcl2[i],d1:d2] #wind
    ssw_wind50[i] = w[:]
    
    rat = ephi_ratio50[sswcl2[i],d1:d2]
    ssw_rat50[i] = rat[:]
    
    cl = ephi_cenlat50[sswcl2[i],d1:d2]
    ssw_cenl50[i] = cl[:]
    
    sz = ephi_size50[sswcl2[i],d1:d2]
    ssw_size50[i] = sz[:]
    
    ep = ephi50[sswcl2[i],d1:d2]
    ssw_ep50[i] = ep[:]

In [8]:
##Specify map dimensions
import cartopy.crs as ccrs
import cartopy.feature as cfeature
lonW = -118
lonE = -66
latS = 24
latN = 50
latRange = np.arange(latS-5,latN+5,.25) # expand the data range a bit beyond the plot range
lonRange = np.arange((lonW-5+360),(lonE+5+360),.25) # Need to match longitude values to those of the coordinate variable

#defining varibales for cartopy plotting
cLat = (latN + latS)/2.
cLon = (lonW + lonE)/2.

map_crs = ccrs.LambertConformal(central_longitude=cLon,
                               central_latitude=cLat) #lat and lon extent
data_crs = ccrs.PlateCarree()
res = '50m'

constrainLon = 7

In [13]:
#####################################################################
#FULL RECORD SLOPE PLOTS

slope = np.zeros((41,82))
pval = np.empty((41,82))
pval[:] = np.nan
    
for i in range(0,41):
    for j in range(0,82):
        reg_temps = temps[:,i,j]
        rt = []
        ep = []
        for k in range(0,488):
            if k <= 56:
                continue
            else:
                t = k-56
                rt.append(reg_temps[k])
                ep.append(wind10[t])
        #print(len(rt))
        #print(len(ep))
        test = scipy.stats.mstats.theilslopes(rt, x= ep,  alpha=0.95)
        t2 = scipy.stats.kendalltau(rt, ep)
        #print(test[0])
        slope[i,j] = test[0]
        if t2[1] <= 0.05:
            pval[i,j] = t2[1]

print(pval)

[[           nan 3.57083102e-02 1.64646242e-02 ... 2.14547795e-08
  2.91227581e-10 3.03689962e-11]
 [           nan            nan            nan ... 2.98218148e-07
  5.25729841e-09 1.62580175e-10]
 [           nan            nan            nan ... 1.59266764e-06
  5.96745097e-08 3.03203514e-09]
 ...
 [2.36259774e-08 3.38808717e-07 4.33689786e-04 ... 1.98584921e-08
  1.51205327e-07 5.93354109e-07]
 [9.87885128e-05            nan 4.41872888e-03 ... 8.38666819e-09
  4.81719163e-08 2.71866830e-07]
 [1.24241157e-02 2.54348154e-09 4.14739195e-16 ... 6.07361711e-09
  1.41027568e-08 4.46702637e-08]]


In [14]:
##plot contour of regression
fig = plt.figure(figsize=(15,15)) #create figure
ax = plt.subplot(1,1,1, projection = map_crs)
ax.set_extent ([lonW,lonE-constrainLon,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

a = ax.contourf(lons[52:],lats[4:45],slope,transform=data_crs, cmap = "bwr") #plot temperature

cs = ax.contourf(lons[52:],lats[4:45], pval, hatches=['..'], colors = 'none',transform=data_crs)

cmap = matplotlib.cm.bwr
norm = matplotlib.colors.Normalize(vmin = np.min(slope),vmax=np.max(slope))

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax,shrink = 0.6) #add colorbar
#tick_locator = ticker.MaxNLocator(nbins=10)
#cbar.locator = tick_locator
#cbar.update_ticks()
cbar.set_label(label='Theilslope', size=18)
cbar.ax.tick_params(labelsize=18)

plt.title("Theilslope of 850hPa Temperature and 10hPa Zonal Mean Wind", fontsize = 18)
plt.show()

plt.savefig("Slope_10hPaWind.png")

In [ ]:
####################################################################################
##NOW FOR THE SSW EVENTS##

In [15]:
#Averaging the appropriate elliptcial diagnostic
##10hPa
srat10 = np.nanmean(ssw_rat10,axis=0)
swind10 = np.nanmean(ssw_wind10,axis=0)
scenl10 = np.nanmean(ssw_cenl10,axis=0)
ssize10 = np.nanmean(ssw_size10,axis=0)
sep10 = np.nanmean(ssw_ep10,axis=0)

##50hPa
srat50 = np.nanmean(ssw_rat50,axis=0)
swind50 = np.nanmean(ssw_wind50,axis=0)
scenl50 = np.nanmean(ssw_cenl50,axis=0)
ssize50 = np.nanmean(ssw_size50,axis=0)
sep50 = np.nanmean(ssw_ep50,axis=0)

temps2 = np.nanmean(ssw_t,axis=0)

In [33]:
slope = np.zeros((41,82))
pval = np.empty((41,82))
pval[:] = np.nan
    
for i in range(0,41):
    for j in range(0,82):
        reg_temps = temps2[:,i,j]
        rt = []
        ep = []
        for k in range(0,81):
            if k <= 56:
                continue
            else:
                t = k-56
                rt.append(reg_temps[k])
                ep.append(swind10[t])
        #print(len(rt))
        #print(len(ep))
        test = scipy.stats.mstats.theilslopes(rt, x= ep,  alpha=0.95)
        t2 = scipy.stats.kendalltau(rt, ep)
        #print(test[0])
        slope[i,j] = test[0]
        if t2[1] <= 0.05:
            pval[i,j] = t2[1]

print(pval)

[[5.03922598e-05 1.72238843e-05 5.40621352e-07 ...            nan
             nan            nan]
 [2.97838236e-05 9.73421202e-06 3.94990103e-06 ...            nan
             nan            nan]
 [8.35123469e-05 5.03922598e-05 9.73421202e-06 ...            nan
             nan            nan]
 ...
 [2.04871982e-12 5.46215947e-11 1.75560024e-10 ...            nan
             nan            nan]
 [9.88310856e-11 9.88310856e-11 1.22654518e-07 ...            nan
             nan            nan]
 [8.27808887e-08 3.88440984e-05 2.08173575e-03 ...            nan
             nan            nan]]


In [ ]:
##plot contour of regression
fig = plt.figure(figsize=(15,15)) #create figure
ax = plt.subplot(1,1,1, projection = map_crs)
ax.set_extent ([lonW,lonE-constrainLon,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

a = ax.contourf(lons[52:],lats[4:45],slope,transform=data_crs, cmap = "bwr") #plot temperature

cs = ax.contourf(lons[52:],lats[4:45], pval, hatches=['..'], colors = 'none',transform=data_crs)


cmap = matplotlib.cm.bwr
norm = matplotlib.colors.Normalize(vmin = np.min(slope),vmax=np.max(slope))

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax,shrink = 0.6) #add colorbar
#tick_locator = ticker.MaxNLocator(nbins=10)
#cbar.locator = tick_locator
#cbar.update_ticks()
cbar.set_label(label='Theilslope', size=18)
cbar.ax.tick_params(labelsize=18)

plt.title("Theilslope of 850hPa Temperature and 10hPa Zonal Mean Wind in SSWs", fontsize = 18)
plt.show()

plt.savefig("Slope_SSW10hPaWind.png")

NameError: name 'plt' is not defined